In [135]:
import flow
from flow.networks.ring import RingNetwork
from flow.core.params import VehicleParams
from flow.controllers.car_following_models import IDMController
from flow.controllers.routing_controllers import ContinuousRouter
from flow.envs.ring.accel import AccelEnv
from flow.core.experiment import Experiment


In [136]:
from flow.controllers.base_lane_changing_controller import BaseLaneChangeController

# We want the ambulance to always prefer the left lane, so we create this lane-changing controller that makes them always enter the left lane
class LeftLaneController(BaseLaneChangeController):
    """A lane-changing model used to move vehicles into the left lane."""
    def get_lane_change_action(self, env):
        current_lane = env.k.vehicle.get_lane(self.veh_id)
        if current_lane < 4:
            return 1
        else:
            return 0

# We want to have vehicles merge to the right if possible when they detect that
# the ambulance is coming up behind them.
class YieldToEmergencyVehicleLaneChangeController(BaseLaneChangeController):
    def get_lane_change_action(self, env):
        current_lane = env.k.vehicle.get_lane(self.veh_id)
        # Is follower an ambulance?
        follower_id = env.k.vehicle.get_follower(self.veh_id)
        if follower_id is not None and follower_id.startswith("ambulance"):
            return -1
        else:
            return 0

In [137]:

vehicles = VehicleParams()
vehicles.add("ambulance",
             acceleration_controller=(IDMController, {}),
             lane_change_controller=(LeftLaneController, {}),
             routing_controller=(ContinuousRouter, {}),
             num_vehicles=1,
             color="red")
vehicles.add("human",
             acceleration_controller=(IDMController, {}),
             lane_change_controller=(YieldToEmergencyVehicleLaneChangeController, {}),
             routing_controller=(ContinuousRouter, {}),
             num_vehicles=22)

In [138]:
from flow.networks.ring import ADDITIONAL_NET_PARAMS
from flow.core.params import NetParams

net_params = NetParams(additional_params={**ADDITIONAL_NET_PARAMS, "lanes": 4})

In [139]:
from flow.core.params import InitialConfig

initial_config = InitialConfig(spacing="uniform", perturbation=1)

In [140]:
from flow.core.params import TrafficLightParams

traffic_lights = TrafficLightParams()

In [141]:
from flow.core.params import SumoParams

sim_params = SumoParams(sim_step=0.1, render=True, emission_path='data')

In [142]:
from flow.envs.ring.accel import ADDITIONAL_ENV_PARAMS
from flow.core.params import EnvParams

env_params = EnvParams(additional_params={**ADDITIONAL_ENV_PARAMS, "horizon": 3000})

In [143]:
flow_params = dict(
    exp_tag='ambulance',
    env_name=AccelEnv,
    network=RingNetwork,
    simulator='traci',
    sim=sim_params,
    env=env_params,
    net=net_params,
    veh=vehicles,
    initial=initial_config,
    tls=traffic_lights,
)

# number of time steps
flow_params['env'].horizon = 3000
exp = Experiment(flow_params)

# run the sumo simulation
_ = exp.run(1, convert_to_csv=True)

human_20


NameError: name 'follower' is not defined